In [1]:
%pip install -q --proxy http://proxy:80 timm

Note: you may need to restart the kernel to use updated packages.


In [5]:
import sys
sys.path.append("../")
from search_space.RegNetTIMM import RegNetCfg, RegNet
import random
import numpy as np
import time
import torch
from IPython.display import clear_output
random_seed = 42
random.seed(random_seed)
# Set seed for NumPy
np.random.seed(random_seed)
# Set seed for PyTorch
torch.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)
# Additional steps if using CuDNN (optional, for GPU acceleration)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [6]:
#regnety_004=RegNetCfg(w0=48, wa=27.89, wm=2.09, group_size=8, depth=16, se_ratio=0.25,downsample=None)

regnety_004=RegNetCfg(w0=24, wa=48, wm=2.4, group_size=8, depth=12, se_ratio=0.25,downsample="avg",preact=False)

In [7]:
regnety_004

RegNetCfg(depth=12, w0=24, wa=48, wm=2.4, group_size=8, bottle_ratio=1.0, se_ratio=0.25, group_min_ratio=0.0, stem_width=32, downsample='conv1x1', linear_out=False, preact=False, num_features=0, act_layer='relu', norm_layer='batchnorm')

In [8]:
model=RegNet(regnety_004,
            in_chans=3,
            num_classes=20,
            #output_stride=8,
            #global_pool='avg',
            drop_rate=0.1,
            drop_path_rate=0.1,
            zero_init_last=True)

[{'out_chs': 24, 'stride': 2, 'dilation': 1, 'depth': 1, 'bottle_ratio': 1.0, 'group_size': 8, 'drop_path_rates': array([0.])}, {'out_chs': 56, 'stride': 2, 'dilation': 1, 'depth': 1, 'bottle_ratio': 1.0, 'group_size': 8, 'drop_path_rates': array([0.00909091])}, {'out_chs': 136, 'stride': 2, 'dilation': 1, 'depth': 2, 'bottle_ratio': 1.0, 'group_size': 8, 'drop_path_rates': array([0.01818182, 0.02727273])}, {'out_chs': 328, 'stride': 2, 'dilation': 1, 'depth': 7, 'bottle_ratio': 1.0, 'group_size': 8, 'drop_path_rates': array([0.03636364, 0.04545455, 0.05454545, 0.06363636, 0.07272727,
       0.08181818, 0.09090909])}, {'out_chs': 800, 'stride': 1, 'dilation': 2, 'depth': 1, 'bottle_ratio': 1.0, 'group_size': 8, 'drop_path_rates': array([0.1])}]


In [5]:
model

RegNet(
  (stem): ConvNormAct(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNormAct2d(
      32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): ReLU(inplace=True)
    )
  )
  (s1): RegStage(
    (b1): Bottleneck(
      (conv1): ConvNormAct(
        (conv): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNormAct2d(
          8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): ReLU(inplace=True)
        )
      )
      (conv2): ConvNormAct(
        (conv): Conv2d(8, 8, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNormAct2d(
          8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): ReLU(inplace=True)
        )
      )
      (se): SEModule(
        (fc1): Conv2d(8, 8, kernel_size=(1, 1), s

In [9]:
import sys
sys.path.append("../")
from evaluation import main
from data_processor import DataProcessor
#from search_space.RegNet import RegNet
import torch
from trainer import Trainer, TrainerDistillation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#from search_space.config import cfg, load_cfg

#Dataset must be in the folder datasets
(train_x, train_y), (valid_x, valid_y), (test_x), metadata = main.load_datasets("AddNIST", truncate=False)

data_processor = DataProcessor(train_x, train_y, valid_x, valid_y, test_x, metadata)
train_loader, valid_loader, test_loader = data_processor.process()

In [9]:
#import timm
#model = timm.create_model('regnety_004', pretrained=False, num_classes=20)

In [10]:
#model

RegNet(
  (stem): ConvNormAct(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNormAct2d(
      32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): ReLU(inplace=True)
    )
  )
  (s1): RegStage(
    (b1): Bottleneck(
      (conv1): ConvNormAct(
        (conv): Conv2d(32, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNormAct2d(
          48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): ReLU(inplace=True)
        )
      )
      (conv2): ConvNormAct(
        (conv): Conv2d(48, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=6, bias=False)
        (bn): BatchNormAct2d(
          48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): ReLU(inplace=True)
        )
      )
      (se): SEModule(
        (fc1): Conv2d(48, 8, kerne

In [9]:
trainer=Trainer(model, device, train_loader, valid_loader, metadata)
model_trained=trainer.train()

0.01
	Epoch   1/2   | Train Acc:  13.44% | Valid Acc:  15.69% | T/Epoch: 45.08s  |
0.005
	Epoch   2/2   | Train Acc:  19.57% | Valid Acc:  20.07% | T/Epoch: 43.97s  |
  Total runtime: 1m,27s


In [7]:
metadata

{'num_classes': 20,
 'input_shape': [50000, 3, 28, 28],
 'codename': 'Adaline',
 'benchmark': 89.85}

In [5]:
metadata["experiment_name"]="tests_one_model/deft_bat"
trainer=TrainerDistillation(model, device, train_loader, valid_loader, metadata)
trainer.train()

[TRAIN] Epoch:1| Time(data):0.000| Time(train):0.057| Loss:2.7805| Top-1:13.240| Top-5:43.726: 100%|██████████| 703/703 [00:58<00:00, 12.08it/s]
[EVAL] Top-1:15.727| Top-5:50.667: 100%|██████████| 235/235 [00:07<00:00, 32.27it/s]
[TRAIN] Epoch:2| Time(data):0.000| Time(train):0.057| Loss:2.5174| Top-1:18.305| Top-5:55.710: 100%|██████████| 703/703 [00:58<00:00, 12.08it/s]
[EVAL] Top-1:19.340| Top-5:56.440: 100%|██████████| 235/235 [00:07<00:00, 33.07it/s]
[TRAIN] Epoch:3| Time(data):0.000| Time(train):0.057| Loss:2.3689| Top-1:21.662| Top-5:60.953: 100%|██████████| 703/703 [00:57<00:00, 12.14it/s]
[EVAL] Top-1:22.873| Top-5:62.360: 100%|██████████| 235/235 [00:07<00:00, 32.91it/s]
[TRAIN] Epoch:4| Time(data):0.000| Time(train):0.057| Loss:2.2570| Top-1:24.487| Top-5:65.525: 100%|██████████| 703/703 [00:57<00:00, 12.17it/s]
[EVAL] Top-1:25.927| Top-5:66.620: 100%|██████████| 235/235 [00:07<00:00, 32.38it/s]
[TRAIN] Epoch:5| Time(data):0.000| Time(train):0.057| Loss:2.1910| Top-1:26.623|

[EVAL] Best accuracy:27.479999542236328


RegNet(
  (stem): ConvNormAct(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNormAct2d(
      32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): ReLU(inplace=True)
    )
  )
  (s1): RegStage(
    (b1): Bottleneck(
      (conv1): ConvNormAct(
        (conv): Conv2d(32, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNormAct2d(
          24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): ReLU(inplace=True)
        )
      )
      (conv2): ConvNormAct(
        (conv): Conv2d(24, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=3, bias=False)
        (bn): BatchNormAct2d(
          24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): ReLU(inplace=True)
        )
      )
      (se): SEModule(
        (fc1): Conv2d(24, 8, kerne

In [ ]:
metadata["experiment_name"]="tests_one_model/deft_bat"
trainer=TrainerDistillation(model, device, train_loader, valid_loader, metadata)
trainer.train()


  0%|          | 0/703 [00:00<?, ?it/s]
[TRAIN] Epoch:1| Time(data):0.000| Time(train):0.059| Loss:2.9848| Top-1:3.125| Top-5:25.000:   0%|          | 0/703 [00:00<?, ?it/s]
[TRAIN] Epoch:1| Time(data):0.000| Time(train):0.050| Loss:2.9899| Top-1:4.688| Top-5:28.906:   0%|          | 1/703 [00:00<01:46,  6.57it/s]
[TRAIN] Epoch:1| Time(data):0.000| Time(train):0.050| Loss:2.9899| Top-1:4.688| Top-5:28.906:   0%|          | 2/703 [00:00<00:54, 12.87it/s]
[TRAIN] Epoch:1| Time(data):0.000| Time(train):0.047| Loss:3.0050| Top-1:3.646| Top-5:26.562:   0%|          | 2/703 [00:00<00:54, 12.87it/s]
[TRAIN] Epoch:1| Time(data):0.000| Time(train):0.046| Loss:3.0204| Top-1:3.906| Top-5:24.219:   0%|          | 3/703 [00:00<00:54, 12.87it/s]
[TRAIN] Epoch:1| Time(data):0.000| Time(train):0.046| Loss:3.0204| Top-1:3.906| Top-5:24.219:   1%|          | 4/703 [00:00<00:49, 14.13it/s]
[TRAIN] Epoch:1| Time(data):0.000| Time(train):0.047| Loss:3.0092| Top-1:4.062| Top-5:25.000:   1%|          | 4/70

[TRAIN] Epoch:1| Time(data):0.000| Time(train):0.055| Loss:3.1024| Top-1:5.705| Top-5:27.085: 100%|██████████| 703/703 [00:55<00:00, 12.65it/s]
[EVAL] Top-1:5.227| Top-5:25.373: 100%|██████████| 235/235 [00:07<00:00, 33.49it/s]
[TRAIN] Epoch:2| Time(data):0.000| Time(train):0.054| Loss:2.9676| Top-1:7.953| Top-5:34.391: 100%|██████████| 703/703 [00:55<00:00, 12.72it/s]
[EVAL] Top-1:10.680| Top-5:43.333: 100%|██████████| 235/235 [00:06<00:00, 34.34it/s]
[TRAIN] Epoch:3| Time(data):0.000| Time(train):0.054| Loss:2.7530| Top-1:11.100| Top-5:44.266: 100%|██████████| 703/703 [00:55<00:00, 12.72it/s]
[EVAL] Top-1:13.207| Top-5:50.213: 100%|██████████| 235/235 [00:06<00:00, 34.27it/s]
[TRAIN] Epoch:4| Time(data):0.000| Time(train):0.054| Loss:2.5970| Top-1:14.369| Top-5:51.643: 100%|██████████| 703/703 [00:55<00:00, 12.70it/s]
[EVAL] Top-1:17.007| Top-5:54.680: 100%|██████████| 235/235 [00:06<00:00, 34.10it/s]
[TRAIN] Epoch:5| Time(data):0.000| Time(train):0.054| Loss:2.5283| Top-1:16.234| To

[EVAL] Best accuracy:17.780000686645508


RegNet(
  (stem): ConvNormAct(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNormAct2d(
      32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): ReLU(inplace=True)
    )
  )
  (s1): RegStage(
    (b1): Bottleneck(
      (conv1): ConvNormAct(
        (conv): Conv2d(32, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNormAct2d(
          24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): ReLU(inplace=True)
        )
      )
      (conv2): ConvNormAct(
        (conv): Conv2d(24, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=3, bias=False)
        (bn): BatchNormAct2d(
          24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): ReLU(inplace=True)
        )
      )
      (se): SEModule(
        (fc1): Conv2d(24, 8, kerne

In [14]:
model.train()

RegNet(
  (stem): ConvNormAct(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNormAct2d(
      32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): ReLU(inplace=True)
    )
  )
  (s1): RegStage(
    (b1): Bottleneck(
      (conv1): ConvNormAct(
        (conv): Conv2d(32, 40, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNormAct2d(
          40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): ReLU(inplace=True)
        )
      )
      (conv2): ConvNormAct(
        (conv): Conv2d(40, 40, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=5, bias=False)
        (bn): BatchNormAct2d(
          40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): ReLU(inplace=True)
        )
      )
      (se): SEModule(
        (fc1): Conv2d(40, 8, kerne